In [1]:
import json

import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import numpy as np
import pandas as pd
from tqdm import tqdm

import lightgbm as lgbm

In [2]:
SUBMISSION_RUN = False

In [3]:
INPUT_BASE_PATH = "/kaggle/input/m5-forecasting-accuracy"
OUTPUT_BASE_BATH = "/kaggle/working"

CALENDAR_DATA = pd.read_csv(f"{INPUT_BASE_PATH}/calendar.csv")
SELL_PRICES = pd.read_csv(f"{INPUT_BASE_PATH}/sell_prices.csv")
SALES_TRAIN_EVALUATION = pd.read_csv(f"{INPUT_BASE_PATH}/sales_train_evaluation.csv")

SAMPLE_SUBMISSION = pd.read_csv(f"{INPUT_BASE_PATH}/sample_submission.csv")
SUBMISSION_INDEX = SAMPLE_SUBMISSION.set_index("id").index
VAL_SUBMISSION = SAMPLE_SUBMISSION[SAMPLE_SUBMISSION["id"].str.contains("validation")]
EVAL_SUBMISSION = SAMPLE_SUBMISSION[SAMPLE_SUBMISSION["id"].str.contains("evaluation")]

In [4]:
# Constants
MAX_TRAIN_TIMESTAMP = 1941
START_TEST_TIMESTAMP = 1942
START_TEST_WM_YR_WK = 11617

# PREPROCESSING & FEATURE ENGINEERING

In [5]:
# Extract categorical mappings
def get_unique_value_id_map(df: pd.DataFrame, col_name: str):
    return {value_id: i for (i, value_id) in enumerate(df[col_name].unique())}


def map_category_ids(
    df: pd.DataFrame,
    column_name: str,
    category_id_map: dict | None = None,
    submission_run: bool = False
) -> pd.DataFrame:
    if category_id_map is None:
        category_id_map = get_unique_value_id_map(df, column_name)
    id_category_map = {v: k for (k, v) in category_id_map.items()}
    df[column_name] = df[column_name].map(category_id_map)
    return (category_id_map, None) if submission_run else (category_id_map, id_category_map)

In [6]:
# Process sales data
def unpivot_sales_df(sales_df: pd.DataFrame, timestamp_cols: list[str]) -> pd.DataFrame:
    sales_df = sales_df.melt(
        id_vars=["item_id", "dept_id", "cat_id", "store_id", "state_id"],
        value_vars=timestamp_cols,
        var_name="d",
        value_name="count"
    )
    sales_df["d"] = sales_df["d"].apply(lambda d: int(d.lstrip("d_")))
    return sales_df


def cast_sales_data_types(sales_df: pd.DataFrame) -> pd.DataFrame:
    category_cols = ["item_id", "dept_id", "cat_id", "store_id", "state_id"]
    for category_col in category_cols:
        sales_df[category_col] = sales_df[category_col].astype("category")
        
    column_downcast_map = [("d", "integer"), ("count", "integer")]
    for (col, dtype) in column_downcast_map:
        sales_df[col] = pd.to_numeric(sales_df[col], downcast=dtype)
    
    return sales_df

In [7]:
# Process calendar data
def select_format_calendar_features(calendar_data: pd.DataFrame) -> pd.DataFrame:
    # Drop columns
    cols_to_drop = ["weekday", "event_name_2", "event_type_2"]
    calendar_data = calendar_data.copy().drop(columns=cols_to_drop)
    
    # Format cols
    calendar_data["d"] = calendar_data["d"].apply(lambda d: int(d.lstrip("d_")))
    calendar_data = calendar_data.rename(columns={
        "event_name_1": "event_name",
        "event_type_1": "event_type",
        "wday": "weekday"
    })
    
    return calendar_data


def cast_calendar_data_types(calendar_df: pd.DataFrame) -> pd.DataFrame:
    category_cols = [
        "weekday",
        "month",
        "year",
        "wm_yr_wk",
        "event_name",
        "event_type",
        "snap_CA",
        "snap_TX",
        "snap_WI"
    ]
    for category_col in category_cols:
        calendar_df[category_col] = calendar_df[category_col].astype("category")
    
    calendar_df["d"] = pd.to_numeric(calendar_df["d"], downcast="integer")
    
    return calendar_df

In [8]:
# Process sell price data
def cast_price_data_types(price_df: pd.DataFrame) -> pd.DataFrame:
    category_cols = ["item_id", "store_id", "wm_yr_wk"]
    for category_col in category_cols:
        price_df[category_col] = price_df[category_col].astype("category")
        
    price_df["sell_price"] = pd.to_numeric(price_df["sell_price"], downcast="float")
    
    return price_df


def extract_state_and_dept_id(sell_prices: pd.DataFrame) -> pd.DataFrame:
    return (
        sell_prices
        .assign(state_id=sell_prices["store_id"].str.extract(r"(^[A-Z]+)"))
        .assign(dept_id=sell_prices["item_id"].str.extract(r"(^[A-Z]+_[0-9]+)"))
    )

In [9]:
# Merge
def merge_sales_calendar_price_data(
    sales_df: pd.DataFrame,
    calendar_df: pd.DataFrame,
    price_df: pd.DataFrame,
    chunk_size: int = 500_000,
) -> pd.DataFrame:
    chunks = []
    for chunk_start in tqdm(range(0, len(sales_df), chunk_size)):
        sales_chunk = sales_df.iloc[chunk_start: chunk_start + chunk_size]
        merged_chunk = (
            sales_chunk.merge(
                right=calendar_df,
                on="d",
                how="left",
            ).merge(
                right=price_df,
                on=["item_id", "store_id", "wm_yr_wk"],
                how="left",
            )
        )
        chunks.append(merged_chunk)
        
#         if chunk_start > 10 * chunk_size: break
            
    return pd.concat(chunks)

In [11]:
# Feature engineering

def fourier_features(data: pd.DataFrame) -> pd.DataFrame:
    data["weekday_sin"] = np.sin((2 * np.pi * data["weekday"].astype(float)) / 7)
    data["weekday_cos"] = np.cos((2 * np.pi * data["weekday"].astype(float)) / 7)
    data["month_sin"] = np.sin((2 * np.pi * data["month"].astype(float)) / 12)
    data["month_cos"] = np.cos((2 * np.pi * data["month"].astype(float)) / 12)
    
    for col in ["weekday_sin", "weekday_cos", "month_sin", "month_cos"]:
        data[col] = pd.to_numeric(data[col], downcast="float")
    
    return data


def av_item_state_prices(data: pd.DataFrame) -> pd.DataFrame:
    av_price = (
        data
        .groupby(["item_id", "state_id", "d"], observed=True)
        [["sell_price"]]
        .mean()
        .rename(columns={"sell_price": "av_item_state_sell_price"})
        .reset_index()
    )
    
    data = data.merge(right=av_price, on=["item_id", "state_id", "d"], how="left")
    data["sell_price"] = pd.to_numeric(data["sell_price"], downcast="float")
    data["av_item_state_sell_price"] = pd.to_numeric(data["av_item_state_sell_price"], downcast="float")
    
    # Fill missing sell prices with av
    data["sell_price"] = data["sell_price"].fillna(data["av_item_state_sell_price"])
    
    return data

In [ ]:
def merge_sales_and_price_data(sales_df: pd.DataFrame, sell_prices: pd.DataFrame) -> pd.DataFrame:
    dedupe_item_state_prices = (
        sell_prices[["item_id", "state_id", "wm_yr_wk", "av_item_state_sell_price"]]
        .drop_duplicates(subset=["item_id", "state_id", "wm_yr_wk"])
    )
    dedupe_dept_state_prices = (
        sell_prices[["dept_id", "state_id", "wm_yr_wk", "av_dept_state_sell_price"]]
        .drop_duplicates(subset=["dept_id", "state_id", "wm_yr_wk"])
    )
    merged = sales_df.merge(
        right=sell_prices[["item_id", "store_id", "wm_yr_wk", "sell_price"]],
        on=["item_id", "store_id", "wm_yr_wk"],
        how="left",
    )
    merged = merged.merge(
        right=dedupe_item_state_prices,
        on=["item_id", "state_id", "wm_yr_wk"],
        how="left",
    )
    merged = merged.merge(
        right=dedupe_dept_state_prices,
        on=["dept_id", "state_id", "wm_yr_wk"],
        how="left",
    )
    return merged


def fill_missing_sell_prices(df: pd.DataFrame) -> pd.DataFrame:
    df["sell_price"] = (
        df["sell_price"]
        .fillna(df["av_item_state_sell_price"])
        .fillna(df["av_dept_state_sell_price"])
    )
    df["av_item_state_sell_price"] = (
        df["av_item_state_sell_price"]
        .fillna(df["av_dept_state_sell_price"])
    )
    return df
    

# EVALUATION DATASET -- PRIVATE LEADERBOARD

In [12]:
# Process sales data

# Map item ids to categories
item_id_map, id_item_map = map_category_ids(SALES_TRAIN_EVALUATION, "item_id", submission_run=SUBMISSION_RUN)
dept_id_map, id_dept_map = map_category_ids(SALES_TRAIN_EVALUATION, "dept_id", submission_run=SUBMISSION_RUN)
cat_id_map, id_cat_map = map_category_ids(SALES_TRAIN_EVALUATION, "cat_id", submission_run=SUBMISSION_RUN)
store_id_map, id_store_map = map_category_ids(SALES_TRAIN_EVALUATION, "store_id", submission_run=SUBMISSION_RUN)
state_id_map, id_state_map = map_category_ids(SALES_TRAIN_EVALUATION, "state_id", submission_run=SUBMISSION_RUN)

# Store category info for each item id
ITEM_ID_CATEGORIES = SALES_TRAIN_EVALUATION[["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"]]

# Convert sales table to long table format
start_t = 1 if SUBMISSION_RUN else 1
timestamp_cols = [f"d_{i}" for i in range(start_t, MAX_TRAIN_TIMESTAMP + 1)]
sales_df = unpivot_sales_df(SALES_TRAIN_EVALUATION, timestamp_cols)

# Cast datatypes
sales_df = cast_sales_data_types(sales_df)

In [13]:
# Process calendar data
calendar_df = select_format_calendar_features(CALENDAR_DATA)

# Map event categories
event_name_map, id_event_name_map = map_category_ids(calendar_df, "event_name", submission_run=SUBMISSION_RUN)
event_type_map, id_event_type_map = map_category_ids(calendar_df, "event_type", submission_run=SUBMISSION_RUN)

# Cast datatypes
calendar_df = cast_calendar_data_types(calendar_df)

In [14]:
# Process price data
item_id_map, id_item_map = map_category_ids(SELL_PRICES, "item_id", item_id_map, SUBMISSION_RUN)
store_id_map, id_store_map = map_category_ids(SELL_PRICES, "store_id", store_id_map, SUBMISSION_RUN)

price_df = cast_price_data_types(SELL_PRICES)

In [15]:
# Merge
data = merge_sales_calendar_price_data(sales_df, calendar_df, price_df)

100%|██████████| 119/119 [04:32<00:00,  2.29s/it]


In [16]:
# Feature engineering
data = fourier_features(data)
data = av_item_state_prices(data)

In [ ]:
def get_item_state_level_sell_prices(sell_prices: pd.DataFrame) -> pd.DataFrame:
    return (
        sell_prices
        .groupby(["item_id", "state_id", "wm_yr_wk"], observed=True)
        [["sell_price"]]
        .mean()
        .rename(columns={"sell_price": "av_item_state_sell_price"})
    )


def get_dept_state_level_sell_prices(sell_prices: pd.DataFrame) -> pd.DataFrame:
    return (
        sell_prices
        .groupby(["dept_id", "state_id", "wm_yr_wk"], observed=True)
        [["sell_price"]]
        .mean()
        .rename(columns={"sell_price": "av_dept_state_sell_price"})
        .reset_index()
    )


def add_average_item_dept_sell_prices(sell_prices: pd.DataFrame) -> pd.DataFrame:
    item_state_prices = get_item_state_level_sell_prices(sell_prices)
    dept_state_prices = get_dept_state_level_sell_prices(sell_prices)
    sell_prices = (
        sell_prices.merge(
            right=item_state_prices,
            on=["item_id", "state_id", "wm_yr_wk"],
            how="left"
        ).merge(
            right=dept_state_prices,
            on=["dept_id", "state_id", "wm_yr_wk"],
            how="left"
        )
    )
    return sell_prices


In [ ]:
item_state_prices = get_item_state_level_sell_prices(data)

In [ ]:
dept_state_prices = get_dept_state_level_sell_prices(data)

In [ ]:
item_state_prices.isna().sum()

In [ ]:
data["sell_price"].isna().sum()

In [ ]:
data.head()

In [ ]:
# Save memory
del sales_df
# price_df = price_df[price_df["wm_yr_wk"] >= START_TEST_WM_YR_WK]

In [ ]:
df = pd.DataFrame({'A': [1, 1, 2, 1, 2], 'B': [np.nan, 2, 3, 4, 5], 'C': [1, 2, 1, 1, 2]}, columns=['A', 'B', 'C'])

In [ ]:
grouper = df.groupby('A')
for group in grouper:
    break

In [ ]:
group

In [ ]:
merged["item_id"].astype(int)

In [ ]:
# Add temporal features
calendar_df = fourier_features(calendar_df)

In [ ]:

price_df = add_average_item_dept_sell_prices(price_df)



# Cast price data
price_df = cast_price_data_types(price_df)

In [ ]:
price_df.head()

In [ ]:
# Merge
data = merge_sales_and_calendar_data(sales_df, calendar_df)

In [ ]:
data = merge_sales_and_price_data(data, price_df)

In [ ]:
data.head()

In [ ]:





# Fill missing price data
data = fill_missing_sell_prices(data)

In [ ]:
# Map categories to intergers
item_id_map, id_item_map = map_category_ids(data, "item_id", SUBMISSION_RUN)
dept_id_map, id_dept_map = map_category_ids(data, "dept_id", SUBMISSION_RUN)
cat_id_map, id_cat_map = map_category_ids(data, "cat_id", SUBMISSION_RUN)
store_id_map, id_store_map = map_category_ids(data, "store_id", SUBMISSION_RUN)
state_id_map, id_state_map = map_category_ids(data, "state_id", SUBMISSION_RUN)


# Set types
data = cast_types(data)

# LightGBM Model

In [ ]:
# Rolling window validation

def time_series_split(
    df: pd.DataFrame,
    n_folds: int = 5,
    horizon: int = 28,
    overlap: int = 0,
    max_timestamp: int = MAX_TIMESTAMP,
):
    min_timestamp = max_timestamp - n_folds * horizon + (n_folds - 1) * overlap
    for fold_idx in range(n_folds):
        start = min_timestamp + fold_idx * (horizon - overlap)
        stop = start + horizon

        train_data = df[df["d"] < start].copy()
        valid_data = df[(df["d"] >= start) & (df["d"] < stop)].copy()
        
        print(f"Fold index: {fold_idx}")
        print(f"Train idx (start, end): ({train_data['d'].min()}, {train_data['d'].max()})")
        print(f"Valid idx (start, end): ({valid_data['d'].min()}, {valid_data['d'].max()})")
        print("==================")
        
        yield train_data, valid_data
        
        
def train_predict_score(
    X_train: pd.DataFrame,
    y_train: pd.Series,
    X_test: pd.DataFrame,
    y_test: pd.Series,
) -> tuple[float, float]:
    train_data = lgbm.Dataset(data=X_train, label=y_train)
    valid_data = lgbm.Dataset(data=X_test, label=y_test)
    model = lgbm.train(
        PARAMETERS,
        num_boost_round=500,
        train_set=train_data,
        valid_sets=[valid_data],
        callbacks=[
            lgbm.early_stopping(stopping_rounds=5),
            lgbm.log_evaluation(period=10),
        ]
    )
    
    y_hat = model.predict(X_test)
    y_hat = np.clip(np.round(y_hat), a_min=0, a_max=np.inf)
    y_hat = y_hat.astype(np.int32)
    rmse = np.sqrt(np.mean((y_test - y_hat) ** 2))

    return rmse, model.best_iteration

In [ ]:
FEATURES = [
    "item_id",
    "dept_id",
    "cat_id",
    "store_id",
    "state_id",
    "weekday",
    "month",
    "year",
    "event_name",
    "event_type",
    "snap_CA",
    "snap_TX",
    "snap_WI",
    "sell_price",
    "av_item_state_sell_price",
    "av_dept_state_sell_price",
]
LABEL = "count"

PARAMETERS = {
    "objective": "poisson",
    "learning_rate": 0.1,
    "metric": "rmse",
    "force_col_wise": True,
}

In [ ]:
if not SUBMISSION_RUN:
    scores = []
    best_iterations = []
    
    for train, valid in tqdm(time_series_split(data, n_folds=5), total=5):
    
        X_train, y_train = train[FEATURES], train[LABEL]
        X_test, y_test = valid[FEATURES], valid[LABEL]
        score, best_iter = train_predict_score(X_train, y_train, X_test, y_test)
        scores.append(score)
        best_iterations.append(best_iter)
    
    cv_results = {
        "scores": scores,
        "mean_score": np.mean(scores),
        "std_score": np.std(scores),
        "best_iterations": best_iterations,
        "mean_best_iteration": np.mean(best_iterations)
    }
    
    with open(f"{OUTPUT_BASE_BATH}/cv_results.json", "w") as f:
        json.dump(cv_results, f)
    
else:
    train_data = lgbm.Dataset(data=data[FEATURES], label=data[LABEL])
    model = lgbm.train(
        PARAMETERS,
        num_boost_round=400,
        train_set=train_data,
    )

# FORECAST & SUBMISSION

In [ ]:
def construct_test_df(
    d_start: int,
    d_end: int,
    item_category_df: pd.DataFrame,
    calendar_df: pd.DataFrame,
    price_df: pd.DataFrame,
) -> pd.DataFrame:
    
    d_range = list(range(d_start, d_end + 1))
    d_range_df = pd.DataFrame(d_range, columns=["d"])
    test_df = pd.merge(item_id_categories, d_range_df, how="cross")
    test_df = merge_sales_and_calendar_data(test_df, calendar_df)
    test_df = merge_sales_and_price_data(test_df, price_df)
    return test_df

def map_test_df_columns(test_df: pd.DataFrame) -> pd.DataFrame:
    test_df["item_id"] = test_df["item_id"].map(item_id_map)
    test_df["dept_id"] = test_df["dept_id"].map(dept_id_map)
    test_df["cat_id"] = test_df["cat_id"].map(cat_id_map)
    test_df["store_id"] = test_df["store_id"].map(store_id_map)
    test_df["state_id"] = test_df["state_id"].map(state_id_map)
    test_df["event_name"] = test_df["event_name"].map(event_name_map)
    test_df["event_type"] = test_df["event_type"].map(event_type_map)
    return test_df

def pivot_forecast_df(X_test: pd.DataFrame, y_hat: np.ndarray) -> pd.DataFrame:
    forecast_df = pd.concat([X_test, pd.DataFrame(y_hat, columns=["count"])], axis=1)
    forecast_df = forecast_df.pivot(columns="d", index="id", values="count").sort_index(axis=1)
    forecast_df = forecast_df.rename_axis(None, axis=1).reset_index()
    return forecast_df

def rename_forecast_columns(forecast_df: pd.DataFrame, forecast_horizon: int = 28) -> pd.DataFrame:
    forecast_df = forecast_df.set_index("id")
    forecast_df.columns = [f"F{i}" for i in range(1, forecast_horizon + 1)]
    return forecast_df.reset_index()

In [ ]:
if SUBMISSION_RUN:
    test_d_start, test_d_end = 1942, 1969
    test_df = construct_test_df(
        d_start=test_d_start,
        d_end=test_d_end,
        item_category_df=item_id_categories,
        calendar_df=calendar_df,
        price_df=price_df,
    )
    test_df = map_test_df_columns(test_df)
    test_df = cast_types(test_df)

    y_hat = model.predict(test_df[FEATURES])
    y_hat = np.clip(np.round(y_hat), a_min=0, a_max=np.inf)
    y_hat = y_hat.astype(np.int32)

    forecast_df = pivot_forecast_df(test_df, y_hat)
    forecast_df = rename_forecast_columns(forecast_df)
    
    # Merge with sample submissions
    EVAL_SUBMISSION = EVAL_SUBMISSION[["id"]].merge(forecast_df, on="id", how="left")
    FINAL_SUBMISSIONS = pd.concat([VAL_SUBMISSION, EVAL_SUBMISSION])
    FINAL_SUBMISSIONS = FINAL_SUBMISSIONS.set_index("id").reindex(SUBMISSION_INDEX).reset_index()
    
    FINAL_SUBMISSIONS.to_csv(f"{OUTPUT_BASE_BATH}/submission.csv", index=False)
    